In [25]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

In [26]:
# ! cp data/Train_target.csv ./raw_data/Train_target.csv
# ! cp data/Test_target.csv ./raw_data/Test_target.csv

In [27]:
train=pd.read_csv("./data/Train.csv")
test=pd.read_csv("./data/Test.csv")
sample_sub=pd.read_csv("./data/sample_sub.csv")

In [43]:
train.head()
train.shape

(15539, 9)

In [ ]:
train.temp[1]

In [ ]:
test.head()

In [28]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
        
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])    

### Features engineering part

In [29]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean) #average
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median) # separe higher from lower values
    x["ptp_"+col_name]=x[col_name].apply(np.ptp) # range of values peak to peak max -min
    return x  


def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]


In [34]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data.columns.tolist()

['ID',
 'location',
 'temp',
 'precip',
 'rel_humidity',
 'wind_dir',
 'wind_spd',
 'atmos_press',
 'target']

In [ ]:
test.shape

In [37]:
len(data.temp[0])

121

In [38]:
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

100%|██████████| 6/6 [00:01<00:00,  5.66it/s]


In [39]:
len(data.temp[0])

16

In [ ]:
data

In [40]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

100%|██████████| 6/6 [00:09<00:00,  1.52s/it]


In [42]:
data.shape

(20574, 51)

In [51]:
for x in range(121): # 121 messdaten pro punkt 
    data["newtemp"+ str(x)] = data.temp.str[x]
    data["newprecip"+ str(x)] = data.precip.str[x]
    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
    data["windspeed"+ str(x)] = data.wind_spd.str[x]
    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]

/var/folders/gz/t1kyprv95tzc2bf6b375z94h0000gn/T/ipykernel_5753/4062778077.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
/var/folders/gz/t1kyprv95tzc2bf6b375z94h0000gn/T/ipykernel_5753/4062778077.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
/var/folders/gz/t1kyprv95tzc2bf6b375z94h0000gn/T/ipykernel_5753/4062778077.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of callin

In [52]:
data.shape

(20574, 777)

In [54]:
features

['temp', 'precip', 'rel_humidity', 'wind_dir', 'wind_spd', 'atmos_press']

In [55]:
data.drop(features,1,inplace=True)

/var/folders/gz/t1kyprv95tzc2bf6b375z94h0000gn/T/ipykernel_5753/3282650053.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data.drop(features,1,inplace=True)


In [56]:
data.shape

(20574, 771)

In [63]:
train=data[data.target.notnull()].reset_index(drop=True)
test=data[data.target.isna()].reset_index(drop=True)

In [65]:
del data  
gc.collect()

1901

In [72]:
train.head()

ID location      target   max_temp   min_temp  mean_temp  \
0     ID_train_0        C   45.126304  27.208333  19.275000  22.299527   
1     ID_train_1        D   79.131702  33.616667  17.983333  24.679063   
2    ID_train_10        A   32.661304  31.841667  18.458333  24.112317   
3   ID_train_100        A   53.850238  27.491667  16.941667  20.845273   
4  ID_train_1000        A  177.418750  28.750000  17.525000  21.870732   

   std_temp   var_temp  median_temp   ptp_temp  ...  newrel_humidity119  \
0  2.594011   6.728895    21.358333   7.933333  ...                 NaN   
1  4.266955  18.206903    23.791667  15.633333  ...            0.649250   
2  3.776377  14.261020    23.304167  13.383333  ...            0.437917   
3  3.080140   9.487259    19.541667  10.550000  ...            0.766417   
4  3.286870  10.803515    20.950000  11.225000  ...                 NaN   

   newwind_dir119  windspeed119  atmospherepressure119  newtemp120  \
0             NaN           NaN                    NaN         NaN   
1       73.528733      1.475833              90.439167   21.450000   
2             NaN           NaN              88.415833         NaN   
3             NaN           NaN              88.490833   24.641667   
4             NaN           NaN                    NaN         NaN   

   newprecip120  newrel_humidity120  newwind_dir120  windspeed120  \
0           NaN                 NaN             NaN           NaN   
1         0.561            0.797333      296.967254      1.019167   
2           NaN                 NaN             NaN           NaN   
3         0.000            0.719667             NaN           NaN   
4           NaN                 NaN             NaN           NaN   

   atmospherepressure120  
0                    NaN  
1              90.529167  
2                    NaN  
3              88.465833  
4                    NaN  

[5 rows x 771 columns]

In [75]:
test.head()

ID location  target   max_temp   min_temp  mean_temp  std_temp  \
0     ID_test_0        C     NaN  26.858333  17.050000  21.389532  2.731833   
1     ID_test_1        C     NaN  26.991667  17.275000  21.302824  2.289268   
2    ID_test_10        D     NaN  29.850000  17.108333  22.144421  2.943047   
3   ID_test_100        C     NaN  27.325000  18.633333  22.361639  2.480003   
4  ID_test_1000        D     NaN  31.091667  16.908333  23.560243  4.136978   

    var_temp  median_temp   ptp_temp  ...  newrel_humidity119  newwind_dir119  \
0   7.462909    20.908333   9.808333  ...            0.725250      167.412340   
1   5.240748    20.716667   9.716667  ...            0.868917      174.138794   
2   8.661528    21.458333  12.741667  ...            0.892833       99.502531   
3   6.150415    21.616667   8.691667  ...            0.897583      329.644351   
4  17.114585    22.825000  14.183333  ...            0.850000       12.645287   

   windspeed119  atmospherepressure119  newtemp120  newprecip120  \
0      1.289167              87.580000   22.766667         0.000   
1      0.665833              87.920000   20.025000         0.000   
2      2.285833              90.409167   20.608333         1.327   
3      0.705833              87.844167   21.483333         0.000   
4      0.373333              90.561667   19.833333         0.000   

   newrel_humidity120  newwind_dir120  windspeed120  atmospherepressure120  
0                 NaN      163.667416      0.774167              87.625833  
1            0.886083      111.558812      0.311667              87.998333  
2            0.910167       68.393016      1.395000              90.464167  
3            0.861333      330.678241      1.026667              87.894167  
4            0.851667       28.090116      0.325000              90.584167  

[5 rows x 771 columns]

#### Cross validation strategy (Kfold,or simple train test split )

In [76]:
Experiment_name="simple_model"

In [79]:
import os
os.makedirs("proc_data", exist_ok=True)
os.makedirs("model_save/lgbm/{}".format(Experiment_name), exist_ok=True) # 
os.makedirs("model_save/catboost/{}".format(Experiment_name), exist_ok=True)
os.makedirs("model_save/xgboost/{}".format(Experiment_name), exist_ok=True)

In [78]:
try : 
    folds=pd.read_csv("./proc_data/folds_id.csv")
    train=train.merge(folds,on="ID",how="left")
    train.fold.nunique()
except : 
    #  you run this cell  only for the first time 
    from sklearn.model_selection import KFold 
    kfold=KFold(n_splits=5,shuffle=True,random_state=2020) # change this random_state or all of you will have the same score  :D 
    train.reset_index(drop=True,inplace=True)
    folds=train[["ID"]].copy()
    folds["fold"]=0
    for fold,(tr_indx,val_ind) in enumerate(kfold.split(folds)) : 
        folds.loc[val_ind,"fold"]=fold
    folds.to_csv("./proc_data/folds_id.csv",index=False)
    train=train.merge(folds,on="ID",how="left")
    
    del folds

### Model 

### Lgbm 

In [81]:
len(train.columns)

772

In [82]:
target_name="target"
id_name="ID"
features_to_remove=[target_name,id_name,"fold","location"]
features=train.columns.tolist()
features=[ fea for fea in  features if fea not in features_to_remove  ]


In [83]:
from sklearn.metrics import mean_squared_error
def metric(y,x):
    return np.sqrt(mean_squared_error(x,y))

In [ ]:
import  lightgbm as lgbm 
import xgboost as xgb 
import catboost as cat 

In [ ]:
def train_function(model,train,test,params,other_params,target_name,features,metric):
    folds_num=train.fold.nunique()
    validation=train[[id_name,"fold",target_name]].copy()
    validation["pred_"+target_name]=0
    sub=test[[id_name]].copy()
    sub[target_name]=0
    for fold in np.sort(train.fold.unique()):
        print("#"*50+" {} ".format(fold)+"#"*50)
        os.makedirs("model_save/lgbm/{}/{}".format(Experiment_name,str(int(fold))), exist_ok=True)
        X_train=train[train.fold!=fold]
        X_val=train[train.fold==fold]
        
        train_pred,validation_pred,test_pred=model(X_train,X_val,test,params,other_params)
        
        validation.loc[validation.fold==fold,"pred_"+target_name]=validation_pred
        sub[target_name]+=test_pred/folds_num
        train_score=metric(X_train[target_name],train_pred)
        val_score=metric(X_val[target_name],validation_pred)
        print("train score : {} validation score : {}".format(round(train_score,4),round(val_score,4)))
    final_validation_score=metric(validation[target_name],validation["pred_"+target_name])
    print("final validation score : {}".format(final_validation_score))
        
    return sub,validation,final_validation_score

def lgbm_model(X_train,X_val,X_test,params,other_params):
    dtrain = lgbm.Dataset(data=X_train[features], label=X_train[target_name], feature_name=features)
    dval = lgbm.Dataset(data=X_val[features], label=X_val[target_name], feature_name=features)

    model = lgbm.train(
        params=params,
        train_set=dtrain,
        num_boost_round=other_params["num_boost_round"],
        valid_sets=(dtrain, dval),
        early_stopping_rounds=other_params["early_stopping_rounds"],
        verbose_eval=other_params["verbose_eval"],
    )        
    best_iteration = model.best_iteration
    train_pred=model.predict(X_train[features], num_iteration=best_iteration)
    validation_pred=model.predict(X_val[features], num_iteration=best_iteration)
    test_pred=model.predict(test[features], num_iteration=best_iteration)
        
    return train_pred,validation_pred,test_pred

In [ ]:
other_params={"num_boost_round":50000000,
              "early_stopping_rounds":50,
              "verbose_eval":1000,
}
lgbm_params = {
    "bagging_fraction": 0.8,
    "bagging_freq": 2,
    "boosting_type": "gbdt",
    "feature_fraction": 0.8,
    "learning_rate": 0.01,
    "max_depth": 8,
    "num_threads": 16,
    "objective": "regression",
    "metric": "rmse",
    "seed": 2020,
}

In [ ]:
sub,validation,score=train_function(model=lgbm_model,
                                    train=train,
                                    test=test,
                                    params=lgbm_params,
                                    other_params=other_params,
                                    target_name=target_name,
                                    features=features,
                                    metric=metric)

In [ ]:
os.makedirs("subs", exist_ok=True)

In [ ]:
sub.to_csv("./subs/lgbm_{}.csv".format(round(score,2)),index=False)